### HW_lesson_3

In [1]:
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#### Задание 1.
Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [2]:
def calc_logloss(y, y_pred):

    for i in range(len(y_pred)):

        if y_pred[i] == 0:
            y_pred[i] += 1e-10
        elif y_pred[i] == 1:
            y_pred[i] -= 1e-10
            
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

#### Задание 2.
Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [3]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [4]:
def robust_scale(X):

    X_rob = X.copy().astype(np.float64)

    for i in range(X.shape[0]):
        if i == 0:
            continue

        percentiles = np.percentile(X[i], (30.0, 80.0))
        X_rob[i] = (X[i] - np.median(X[i])) / percentiles[1] - percentiles[0]

    return X_rob

In [5]:
X_rob = robust_scale(X)
X_rob

array([[ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00],
       [-1.14705882e+00, -1.14705882e+00, -8.52941176e-01,
        -1.14705882e+00, -5.58823529e-01, -1.44117647e+00,
         2.94117647e-02,  1.50000000e+00, -1.14705882e+00,
        -8.52941176e-01],
       [-6.70188356e+02, -6.70051370e+02, -6.70017123e+02,
        -6.70119863e+02, -6.69537671e+02, -6.69982877e+02,
        -6.69503425e+02, -6.69160959e+02, -6.70222603e+02,
        -6.69845890e+02],
       [-1.22727273e+00, -1.22727273e+00, -7.72727273e-01,
        -1.22727273e+00, -7.72727273e-01, -1.22727273e+00,
        -3.18181818e-01, -3.18181818e-01, -1.22727273e+00,
        -7.72727273e-01]])

In [6]:
def sigmoid(z):
    result = 1 / (1 + np.exp(-z))
    return result

In [7]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    #if i % (iterations / 10) == 0:
        #print(i, W, err)
    return (i, W, err)

In [8]:
W = eval_model(X_rob, y, iterations=1000, alpha=1e-5)
W

(1000,
 array([ 4.97682736e-01, -1.39483974e-01, -1.18747032e-03,  1.52252378e+00]),
 0.6138006623424344)

In [9]:
def gradient_descent_optimal_alpha(X, y, epsilon):
    err_border = float('inf')
    for alpha in np.linspace(1e-7, 1e-2, num = 1000):
        i, W, err = eval_model(X, y, iterations=1000, alpha=alpha)
        if (err < err_border):
            i_border = i
            W_border = W
            err_border = err
            alpha_border = alpha
    print(f"Заданная точность: {epsilon}\nКоличество итераций = {i_border}\nСкорость обучения = {alpha_border}\nW = {W_border} \nlogloss = {err_border}")
    return W_border

In [10]:
W1 = gradient_descent_optimal_alpha(X_rob, y, 1e-4)

Заданная точность: 0.0001
Количество итераций = 1000
Скорость обучения = 1.0109909909909911e-05
W = [ 4.97682734e-01 -1.39491656e-01 -1.18746688e-03  1.52252710e+00] 
logloss = 0.6138000261349721


#### Задание 3.
Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [11]:
def calc_pred_proba(W, X):
    y_pred_proba = 1 / (1 + np.exp(-np.dot(W, X)))
    return y_pred_proba

In [12]:
calc_pred_proba(W1,X_rob)

array([0.39770385, 0.39766489, 0.55867488, 0.39768437, 0.5483954 ,
       0.40751238, 0.69086209, 0.64533853, 0.39771359, 0.55862474])

#### Задание 4.
Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [13]:
def calc_pred(W, X, border=0.6):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred_proba = y_pred_proba.copy()
    y_pred_proba[y_pred_proba >= border] = 1
    y_pred_proba[y_pred_proba < border] = 0
    return y_pred_proba

In [14]:
Y_pred = calc_pred(W1, X_rob, border=0.6)
Y_pred

array([0., 0., 0., 0., 0., 0., 1., 1., 0., 0.])

#### Задание 5.
Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [15]:
def accuracy(y, y_pred):
    res = (y_pred == y).sum()/len(y)
    return res

In [16]:
accuracy(y, Y_pred)

0.5

In [17]:
def error_matrix(y, y_pred):
    TP = y[(y - y_pred) == 0].sum()
    FP = ((y - y_pred) == -1).sum()
    FN = ((y - y_pred) == 1).sum()
    TN = (y[(y - y_pred) == 0]==0).sum()
    
    return np.array([[TP, FP], 
                     [FN, TN]])

In [18]:
EM = error_matrix(y, Y_pred)
EM

array([[1., 1.],
       [4., 4.]])

In [19]:
def precision_recall_F1(y, y_pred):
    TP = EM[0,0]
    FP = EM[0,1]
    FN = EM[1,0]
    TN = EM[1,1]
    
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    F1 = 2 * precision * recall / (precision + recall)
    
    return precision, recall, F1

In [20]:
precision_recall_F1(y, Y_pred)

(0.5, 0.2, 0.28571428571428575)

#### Задание 6.
Могла ли модель переобучиться? Почему?

Модель могла переобучиться т.к. в модели классификации признаки могут пересекаться, что приводит к переобучению модели.